# **EBAC/Regressão II: Regressão Múltipla**



## **Tarefa I: Previsão de renda II**

####

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [7]:
# Importação das bibliotecas
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import mean_squared_error

%matplotlib inline

## **1. Carregando o dado**

In [8]:
df = pd.read_csv('previsao_de_renda.csv')

## **1.1 Exploração e Limpeza**

In [9]:
print("Descritiva Inicial\n")
df.info()

Descritiva Inicial

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-n

In [10]:
df = df.drop(['Unnamed: 0','data_ref','id_cliente'], axis = 1)

print("Descritiva Após a Limpeza\n")
df.info()

Descritiva Após a Limpeza

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   15000 non-null  object 
 1   posse_de_veiculo       15000 non-null  bool   
 2   posse_de_imovel        15000 non-null  bool   
 3   qtd_filhos             15000 non-null  int64  
 4   tipo_renda             15000 non-null  object 
 5   educacao               15000 non-null  object 
 6   estado_civil           15000 non-null  object 
 7   tipo_residencia        15000 non-null  object 
 8   idade                  15000 non-null  int64  
 9   tempo_emprego          12427 non-null  float64
 10  qt_pessoas_residencia  15000 non-null  float64
 11  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 1.2+ MB


#### **Ex. 1) Separe a base em treinamento e teste (25% para teste, 75% para treinamento)**

In [12]:
df_train, df_test = train_test_split(df,test_size = 0.25, random_state =42) 

#### **Ex. 2) 2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?** 

In [17]:
# Criando o modelo a apartir dos dados

eq1 = ''' renda ~ sexo
                   + posse_de_veiculo
                   + posse_de_imovel
                   + qtd_filhos
                   + tipo_renda
                   + educacao
                   + estado_civil
                   + tipo_residencia
                   + idade
                   + tempo_emprego
                   + qt_pessoas_residencia
                   '''
y_train, X_train= patsy.dmatrices(formula_like=eq1, data=df_train)
y_test, X_test = patsy.dmatrices(formula_like=eq1, data=df_test)

In [18]:
## Ridge e Lasso

alphas = [0,0.001,0.005,0.01,0.05,0.1]
r2_ridge = []
r2_lasso = []

for alpha in alphas:
    modelo = sm.OLS(y_train, X_train)
    reg_ridge = modelo.fit_regularized(method='elastic_net',
                                     refit=True,
                                     L1_wt=0, #ridge
                                     alpha = alpha)
    
    reg_lasso = modelo.fit_regularized(method='elastic_net',
                                     refit=True,
                                     L1_wt=1, #Lasso
                                     alpha = alpha)
    y_pred_ridge = reg_ridge.predict(X_test)
    y_pred_lasso = reg_lasso.predict(X_test)
    aux = r2_score(y_test, y_pred_ridge)
    tmp = r2_score(y_test, y_pred_lasso)
    
    r2_ridge.append(aux)
    r2_lasso.append(tmp)



In [19]:
tab = pd.DataFrame({'alpha':alphas,
              'R2 (Ridge)': r2_ridge,  'R2 (Lasso)': r2_lasso})

tab

,alpha,R2 (Ridge),R2 (Lasso)
0,0.000,0.262434,0.262434
1,0.001,0.262262,0.262434
2,0.005,0.261966,0.262434
3,0.010,0.261568,0.262434
4,0.050,0.256383,0.262500
5,0.100,0.248634,0.262434


In [21]:
# Encontrando o melhor R2 para Ridge
best_r2_ridge = tab.loc[tab['R2 (Ridge)'].idxmax()]['R2 (Ridge)']

# Encontrando o melhor R2 para Lasso
best_r2_lasso = tab.loc[tab['R2 (Lasso)'].idxmax()]['R2 (Lasso)']

print("Melhor R2 para Ridge:")
print(best_r2_ridge)

print("\nMelhor R2 para Lasso:")
print(best_r2_lasso)

Melhor R2 para Ridge:
0.26243403442818514

Melhor R2 para Lasso:
0.2625001021319665


In [24]:

print("Entre os dois o Melhor foi o Lasso com")
print(best_r2_lasso)

Entre os dois o Melhor foi o Lasso com
0.2625001021319665


Mas se considerarmos duas casas decimais não exsitem diferenças significativas entre eles.

#### **Ex. 3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?**


Feito junto com **Ex.2** 


#### **Ex. 4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?**


In [25]:

df_dummies = pd.get_dummies(data=df.dropna(), drop_first=True).astype(int)

X = df_dummies.iloc[:, :-1]
y = df_dummies.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [28]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out=0.05, 
                       verbose=True):
    
    included = list(initial_list)
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(dtype=float, index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Adicione  {:30} com p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()  # null if pvalues is empty
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
            if verbose:
                print('Retire {:30} com p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

results = stepwise_selection(X_test, y_test)

Adicione  sexo_M                         com p-value 9.62952e-69
Adicione  idade                          com p-value 5.4702e-26


In [29]:
X_step = df_dummies[results]
y = df_dummies['renda']
X_train, X_test, y_train, y_test = train_test_split(X_step, y, test_size=0.25, random_state=42)

y_pred = sm.OLS(y_train, X_train).fit().predict(X_test)
print(f'R2: {r2_score(y_test, y_pred)}')

R2: 0.12244938441873898


In [30]:
## Ridge e Lasso

alphas = [0,0.001,0.005,0.01,0.05,0.1]
r2_ridge = []
r2_lasso = []

for alpha in alphas:
    modelo = sm.OLS(y_train, X_train)
    reg_ridge = modelo.fit_regularized(method='elastic_net',
                                     refit=True,
                                     L1_wt=0, #ridge
                                     alpha = alpha)
    
    reg_lasso = modelo.fit_regularized(method='elastic_net',
                                     refit=True,
                                     L1_wt=1, #Lasso
                                     alpha = alpha)
    y_pred_ridge = reg_ridge.predict(X_test)
    y_pred_lasso = reg_lasso.predict(X_test)
    aux = r2_score(y_test, y_pred_ridge)
    tmp = r2_score(y_test, y_pred_lasso)
    
    r2_ridge.append(aux)
    r2_lasso.append(tmp)


tab = pd.DataFrame({'alpha':alphas,
              'R2 Stepwise (Ridge)': r2_ridge,  'R2 Stepwise (Lasso)': r2_lasso})

tab


,alpha,R2 Stepwise (Ridge),R2 Stepwise (Lasso)
0,0.000,0.122449,0.122449
1,0.001,0.122476,0.122449
2,0.005,0.122548,0.122449
3,0.010,0.122562,0.122449
4,0.050,0.120542,0.122449
5,0.100,0.115369,0.122449


In [31]:
tab.describe()

,alpha,R2 Stepwise (Ridge),R2 Stepwise (Lasso)
count,6.000000,6.000000,6.000000
mean,0.027667,0.120991,0.122449
std,0.040083,0.002865,0.000000
min,0.000000,0.115369,0.122449
25%,0.002000,0.121019,0.122449
50%,0.007500,0.122463,0.122449
75%,0.040000,0.122530,0.122449
max,0.100000,0.122562,0.122449



#### **Ex. 5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?**


<h4 style="color:green">No loop executado anteriormente, fica nítido que o modelo Stepwise para Lasso apresenta um desvio padrão nulo para o R quadrado, permanecendo o mesmo para todos os valores de alpha. Já o modelo stepwise para Ridge consegue variar o R quadrado conforme as diferentes iterações de alpha, além de apresentar maior valor </h4>


#### **Ex. 6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.**


Aqui é apar ser criativo é criar uma nova e2

In [38]:
# Ajustando o modelo de regressão linear com log da renda como variável dependente
eq2 = ''' np.log(renda) ~ sexo
                   + posse_de_veiculo
                   + posse_de_imovel
                   + qtd_filhos
                   + tipo_renda
                   + educacao
                   + estado_civil
                   + tipo_residencia
                   + idade
                   + tempo_emprego
                   + qt_pessoas_residencia
                   '''
y_train, X_train= patsy.dmatrices(formula_like=eq2, data=df_train)
y_test, X_test = patsy.dmatrices(formula_like=eq2, data=df_test)

modelo = sm.OLS(y_train, X_train)

reg = modelo.fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     214.0
Date:                Sun, 09 Jun 2024   Prob (F-statistic):               0.00
Time:                        10:27:43   Log-Likelihood:                -10180.
No. Observations:                9313   AIC:                         2.041e+04
Df Residuals:                    9288   BIC:                         2.059e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            6.6594      0.271     24.577      0.000       6.128       7.191
sexo[T.M]                            0.7830      0.017     45.956      0.000       0.750       0.816
posse_de_veiculo[T.True]             0.0469      0.016      2.857      0.004       0.015       0.079
posse_de_imovel[T.True]              0.0802      0.016      4.959      0.000       0.049       0.112
tipo_renda[T.Bolsista]               0.2451      0.296      0.829      0.407      -0.335       0.825
tipo_renda[T.Empresário]             0.1645      0.017      9.535      0.000       0.131       0.198
tipo_renda[T.Pensionista]           -0.4524      0.362     -1.250      0.211      -1.162       0.257
tipo_renda[T.Servidor público]       0.0600      0.026      2.294      0.022       0.009       0.111
educacao[T.Pós graduação]            0.1777      0.194      0.917      0.359      -0.202       0.558
educacao[T.Secundário]              -0.0205      0.082     -0.250      0.803      -0.181       0.140
educacao[T.Superior completo]        0.0906      0.082      1.100      0.271      -0.071       0.252
educacao[T.Superior incompleto]     -0.0776      0.089     -0.871      0.384      -0.252       0.097
estado_civil[T.Separado]             0.2647      0.128      2.065      0.039       0.013       0.516
estado_civil[T.Solteiro]             0.2178      0.125      1.736      0.083      -0.028       0.464
estado_civil[T.União]               -0.0140      0.029     -0.478      0.633      -0.071       0.043
estado_civil[T.Viúvo]                0.3126      0.133      2.352      0.019       0.052       0.573
tipo_residencia[T.Casa]              0.0071      0.065      0.111      0.912      -0.119       0.134
tipo_residencia[T.Com os pais]       0.0437      0.072      0.610      0.542      -0.097       0.184
tipo_residencia[T.Comunitário]      -0.0535      0.120     -0.447      0.655      -0.288       0.181
tipo_residencia[T.Estúdio]           0.1710      0.113      1.508      0.132      -0.051       0.393
tipo_residencia[T.Governamental]    -0.0007      0.078     -0.008      0.993      -0.153       0.151
qtd_filhos                          -0.2107      0.124     -1.693      0.090      -0.455       0.033
idade                                0.0050      0.001      5.399      0.000       0.003       0.007
tempo_emprego                        0.0611      0.001     51.148      0.000       0.059       0.063
qt_pessoas_residencia                0.2359      0.124      1.900      0.057      -0.008       0.479
==============================================================================
Omnibus:                        2.308   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.315   Jarque-Bera (JB):                2.325
Skew:             

<h4 style="color:green">O desempenho do modelo melhora consideravelmente ao utilizarmos o log de renda como variável dependente, apresentando um valor para o R quadrado de 0.356 </h4>


#### **Ex. 7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.**

In [42]:
eq2 = ''' renda ~ sexo
                   + posse_de_veiculo
                   + posse_de_imovel
                   + qtd_filhos
                   + tipo_renda
                   + educacao
                   + estado_civil
                   + tipo_residencia
                   + idade
                   + tempo_emprego
                   + qt_pessoas_residencia
                   '''

y, X = patsy.dmatrices(
    eq2,
    df
)

y = y.ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=104)
tree1 = DecisionTreeRegressor(max_depth=7)
tree1.fit(X_train, y_train)

y_pred = tree1.predict(X_test)

results_df_tree = pd.DataFrame({
    'Actual': y_test,
    'y_pred': y_pred
})

In [43]:
r_squared = tree1.score(X_test, y_test)
r_squared

0.4465286572782352

<h4 style="color:green">Em geral, o R quadrado se apresenta bem maior no modelo de árvore de regressão</h4>